## Importamos las bibliotecas

In [ ]:
import os
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tensorboardX import SummaryWriter
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as scio


## Configuración de datos (data_utils.py)

In [ ]:
def is_image_file(filename):
    return any(filename.endswith(extension) for extension in ['.mat'])

Definición de la clase TrainsetFromFolder

In [ ]:
class TrainsetFromFolder(data.Dataset):
    def __init__(self, dataset_dir):
        super(TrainsetFromFolder, self).__init__()
        self.image_filenames = [join(dataset_dir, x) for x in listdir(dataset_dir) if is_image_file(x)]

Se utiliza para manejar el conjunto de datos de entrenamiento.

In [ ]:
    def __getitem__(self, index):
        mat = scio.loadmat(self.image_filenames[index], verify_compressed_data_integrity=False)
        input = mat['lr'].astype(np.float32)
        label = mat['hr'].astype(np.float32)
        
        return torch.from_numpy(input), torch.from_numpy(label)
        
    def __len__(self):
        return len(self.image_filenames)

Carga un archivo .mat y extrae dos variables:
- lr (baja resolución) se almacena como input.
- hr (alta resolución) se almacena como label.

### Definición de la clase ValsetFromFolder

In [ ]:
class ValsetFromFolder(data.Dataset):
    def __init__(self, dataset_dir):
        super(ValsetFromFolder, self).__init__()
        self.image_filenames = [join(dataset_dir, x) for x in listdir(dataset_dir) if is_image_file(x)]

Está orientada al conjunto de validación

### Función __getitem__

    def __getitem__(self, index):
        mat = scio.loadmat(self.image_filenames[index])
        input = mat['LR'].astype(np.float32).transpose(2, 0, 1)
        label = mat['HR'].astype(np.float32).transpose(2, 0, 1)
        
        return torch.from_numpy(input).float(), torch.from_numpy(label).float()

Carga los datos de validación y ajusta las matrices (input y label).

In [ ]:
    def __len__(self):
        return len(self.image_filenames)

## Configuración de entrenamiento (option.py)

In [ ]:
import argparse

class Options:
    def __init__(self):
        parser = argparse.ArgumentParser(description="Super-Resolution")
        parser.add_argument("--upscale_factor", default=4, type=int, help="super resolution upscale factor")
        parser.add_argument('--seed', type=int, default=1, help='random seed (default: 1)')
        parser.add_argument("--batchSize", type=int, default=16, help="training batch size")
        parser.add_argument("--nEpochs", type=int, default=100, help="maximum number of epochs to train")
        parser.add_argument("--show", action="store_true", help="show Tensorboard")
        parser.add_argument("--lr", type=float, default=1e-4, help="learning rate")
        parser.add_argument("--cuda", action="store_true", help="Use cuda")
        parser.add_argument("--gpus", default="0,1,2,3", type=str, help="gpu ids (default: 0)")
        parser.add_argument("--threads", type=int, default=12, help="number of threads for dataloader to use")
        parser.add_argument("--resume", default="", type=str, help="Path to checkpoint (default: none)")
        parser.add_argument("--start-epoch", default=1, type=int, help="Manual epoch number (useful on restarts)")
        parser.add_argument("--datasetName", default="CAVE", type=str, help="data name")
        parser.add_argument('--n_module', type=int, default=5, help='number of modules')
        parser.add_argument('--n_feats', type=int, default=32, help='number of feature maps')
        parser.add_argument('--model_name', default='checkpoint_x4_n/model_4_epoch_100.pth', type=str, help='model name')
        parser.add_argument('--method', default='SFCSR', type=str, help='method name')
        
        self.opt = parser.parse_args([])

opt = Options().opt


## Arquitectura del modelo (model.py)

### Bloque de Atención de Canal (ChannelAttention)

In [ ]:
class ChannelAttention(nn.Module):
    def __init__(self, n_feats, ratio=8):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool3d(1)
        self.max_pool = nn.AdaptiveMaxPool3d(1)
        self.fc1 = nn.Conv3d(n_feats, n_feats // ratio, kernel_size=1, bias=False)
        self.fc2 = nn.Conv3d(n_feats // ratio, n_feats, kernel_size=1, bias=False)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc2(self.relu(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu(self.fc1(self.max_pool(x))))
        return self.sigmoid(avg_out + max_out) * x

- Usa un AdaptiveAvgPool3d con salida de tamaño (1, 1, 1), calculando el promedio de cada canal en un tensor de tres dimensiones
- fc1 y fc2: son capas convolucionales de 3D que implementan una red de atención.
- relu: aplicada después de fc1, introduce no linealidad.
- sigmoid: aplicada al final, genera una máscara de atención con valores entre 0 y 1 para ponderar la importancia de cada canal.
- avg_out y max_out, combinan la información promedio y máxima para obtener una máscara final de atención.

### Bloque CBAM simplificado (solo canal)

In [ ]:
class CBAM(nn.Module):
    def __init__(self, n_feats, ratio=8):
        super(CBAM, self).__init__()
        self.channel_attention = ChannelAttention(n_feats, ratio)

    def forward(self, x):
        x = self.channel_attention(x)
        return x

Este bloque simplificado CBAM solo aplica atención de canal, usando el bloque ChannelAttention.

### Clase TwoCNN con CBAM

In [ ]:
class TwoCNN(nn.Module):
    def __init__(self, wn, n_feats=64): 
        super(TwoCNN, self).__init__()
        self.body = wn(nn.Conv2d(n_feats, n_feats, kernel_size=(3,3), stride=1, padding=(1,1), bias=False))
        self.cbam = CBAM(n_feats)
        self.adjust_channels = None

    def forward(self, x):
        out = self.body(x)
        out = self.cbam(out.unsqueeze(2)).squeeze(2) 

        # Ajusta canales si `x` y `out` no coinciden
        if out.shape[1] != x.shape[1]:
            self.adjust_channels = nn.Conv2d(out.shape[1], x.shape[1], kernel_size=1).to(out.device)
            out = self.adjust_channels(out)
        
        out = torch.add(out, x)
        return out  

- Una convolución Conv2d con un kernel de 3x3, stride de 1 y padding de 1, que mantiene la dimensión espacial de la entrada.
- Un bloque de atención canal-espacial.
- adjust_channels: se utiliza para ajustar dinámicamente la cantidad de canales en caso de que los canales de out y x no coincidan.
- unsqueeze(2) añade una dimensión extra en el eje 2, que es necesario si CBAM espera datos en formato tridimensional, y luego squeeze(2) elimina esta dimensión después de aplicar la atención.
- adjust_channels se define como una capa Conv2d con un kernel_size=1, que ajusta la cantidad de canales de out para que coincida con x.

### Clase ThreeCNN con CBAM

In [ ]:
class ThreeCNN(nn.Module):
    def __init__(self, wn, n_feats=64):
        super(ThreeCNN, self).__init__()
        self.act = nn.ReLU(inplace=True)
        body_spatial = [wn(nn.Conv3d(n_feats, n_feats, kernel_size=(1,3,3), stride=1, padding=(0,1,1), bias=False)) for _ in range(2)]
        body_spectral = [wn(nn.Conv3d(n_feats, n_feats, kernel_size=(3,1,1), stride=1, padding=(1,0,0), bias=False)) for _ in range(2)]
        
        self.body_spatial = nn.Sequential(*body_spatial)
        self.body_spectral = nn.Sequential(*body_spectral)
        self.cbam = CBAM(n_feats)
        self.adjust_channels = None

    def forward(self, x): 
        out = x
        for i in range(2):  
            out_spatial = self.body_spatial[i](out)
            out_spectral = self.body_spectral[i](out)
            out = torch.add(out_spatial, out_spectral)
            if i == 0:
                out = self.act(out)
        
        out = self.cbam(out)
        
        # Ajusta canales de `out` a `x` dinámicamente si no coinciden
        if out.shape[1] != x.shape[1]:
            self.adjust_channels = nn.Conv3d(out.shape[1], x.shape[1], kernel_size=1).to(out.device)
            out = self.adjust_channels(out)
        
        if out.shape == x.shape:
            out = torch.add(out, x)
        else:
            print(f"Dimension mismatch before final addition: out {out.shape}, x {x.shape}")
            return None
        
        return out

- body_spatial realiza convoluciones con un kernel (1, 3, 3), enfocándose en las dimensiones espaciales sin alterar la dimensión espectral. 
- body_spectral usa un kernel (3, 1, 1), procesando las dimensiones espectrales sin modificar las dimensiones espaciales. 
- cbam es la capa de atención canal-espacial (CBAM) que refuerza las características más importantes en la salida conjunta de body_spatial y body_spectral.
forward:
- out_spatial procesa out usando la i-ésima capa de body_spatial.
- out_spectral procesa out con la i-ésima capa de body_spectral.
- Se suman los resultados out_spatial y out_spectral, y el resultado se almacena en out.
En la primera iteración, out pasa por self.act, aplicando la función de activación ReLU.
- Después de las iteraciones, out pasa por el bloque CBAM para recalibrar la atención de canal y espacial.

### Clase principal SFCSR

In [ ]:
class SFCSR(nn.Module):
    def __init__(self, args):
        super(SFCSR, self).__init__()
        scale = args.upscale_factor
        n_feats = args.n_feats
        self.n_module = args.n_module        
                 
        wn = lambda x: torch.nn.utils.weight_norm(x)
    
        self.gamma_X = nn.Parameter(torch.ones(self.n_module)) 
        self.gamma_Y = nn.Parameter(torch.ones(self.n_module)) 
        self.gamma_DFF = nn.Parameter(torch.ones(4))
        self.gamma_FCF = nn.Parameter(torch.ones(2))
        
        ThreeHead = [wn(nn.Conv3d(1, n_feats, kernel_size=(1,3,3), stride=1, padding=(0,1,1), bias=False)),
                     wn(nn.Conv3d(n_feats, n_feats, kernel_size=(3,1,1), stride=1, padding=(1,0,0), bias=False))]
        self.ThreeHead = nn.Sequential(*ThreeHead)

        TwoHead = [wn(nn.Conv2d(1, n_feats, kernel_size=(3,3),  stride=1, padding=(1,1), bias=False))]
        self.TwoHead = nn.Sequential(*TwoHead)

        TwoTail = []
        if (scale & (scale - 1)) == 0: 
            for _ in range(int(math.log(scale, 2))):
                TwoTail.append(wn(nn.Conv2d(n_feats, n_feats*4, kernel_size=(3,3), stride=1, padding=(1,1), bias=False)))
                TwoTail.append(nn.PixelShuffle(2))           
        else:
            TwoTail.append(wn(nn.Conv2d(n_feats, n_feats*9, kernel_size=(3,3), stride=1, padding=(1,1), bias=False)))
            TwoTail.append(nn.PixelShuffle(3))  
        TwoTail.append(wn(nn.Conv2d(n_feats, 1, kernel_size=(3,3),  stride=1, padding=(1,1), bias=False)))                                 	    	
        self.TwoTail = nn.Sequential(*TwoTail)

        self.twoCNN = nn.Sequential(*[TwoCNN(wn, n_feats) for _ in range(self.n_module)])
        self.reduceD_Y = wn(nn.Conv2d(n_feats*self.n_module, n_feats, kernel_size=(1,1), stride=1, bias=False))                          
        self.twofusion = wn(nn.Conv2d(n_feats, n_feats, kernel_size=(3,3),  stride=1, padding=(1,1), bias=False))

        self.threeCNN = nn.Sequential(*[ThreeCNN(wn, n_feats) for _ in range(self.n_module)])
        self.reduceD = nn.Sequential(*[wn(nn.Conv2d(n_feats*4, n_feats, kernel_size=(1,1), stride=1, bias=False)) for _ in range(self.n_module)])                              
        self.reduceD_X = wn(nn.Conv3d(n_feats*self.n_module, n_feats, kernel_size=(1,1,1), stride=1, bias=False))
        
        threefusion = [wn(nn.Conv3d(n_feats, n_feats, kernel_size=(1,3,3), stride=1, padding=(0,1,1), bias=False)),
                       wn(nn.Conv3d(n_feats, n_feats, kernel_size=(3,1,1), stride=1, padding=(1,0,0), bias=False))]
        self.threefusion = nn.Sequential(*threefusion)

        self.reduceD_DFF = wn(nn.Conv2d(n_feats*4, n_feats, kernel_size=(1,1), stride=1, bias=False))  
        self.conv_DFF = wn(nn.Conv2d(n_feats, n_feats, kernel_size=(1,1), stride=1, bias=False)) 
        self.reduceD_FCF = wn(nn.Conv2d(n_feats*2, n_feats, kernel_size=(1,1), stride=1, bias=False))  
        self.conv_FCF = wn(nn.Conv2d(n_feats, n_feats, kernel_size=(1,1), stride=1, bias=False))    


La red cuenta con dos partes, ThreeHead y TwoHead, que procesan las características de entrada en el dominio tridimensional y bidimensional, respectivamente.
- La parte final de la red, TwoTail, se encarga de ajustar la salida para lograr el escalado deseado (scale).
- twoCNN: Cada TwoCNN incluye una capa de convolución y un módulo de atención CBAM para resaltar características importantes.
- twoFusion: Combina las características procesadas por los módulos twoCNN.
- threeCNN: Módulo que procesa tanto información espacial como espectral en el dominio tridimensional.
- threefusion: capa convolucional en 3D que combina la información procesada por los módulos threeCNN.

La red incluye varias capas de reducción de dimensión (reduce) en 2D y 3D, que permiten ajustar el número de canales después de cada bloque de procesamiento
Las capas conv_DFF y conv_FCF son capas convolucionales finales. Estas capas aseguran que las características importantes se preserven y se resalten en la imagen generada

### Forward SFCSR

In [ ]:
    def forward(self, x, y, localFeats, i):
        x = x.unsqueeze(1)     
        x = self.ThreeHead(x)    
        skip_x = x         

        y = y.unsqueeze(1)
        y = self.TwoHead(y)
        skip_y = y

        channelX = []
        channelY = []        

        for j in range(self.n_module):        
            x = self.threeCNN[j](x)    
            x = torch.add(skip_x, x)          
            channelX.append(self.gamma_X[j]*x)

            y = self.twoCNN[j](y)           
            y = torch.cat([y, x[:,:,0,:,:], x[:,:,1,:,:], x[:,:,2,:,:]],1)
            y = self.reduceD[j](y)      
            y = torch.add(skip_y, y)         
            channelY.append(self.gamma_Y[j]*y) 
                              
        x = torch.cat(channelX, 1)
        x = self.reduceD_X(x)
        x = self.threefusion(x)
      	                
        y = torch.cat(channelY, 1)        
        y = self.reduceD_Y(y) 
        y = self.twofusion(y)        
     
        y = torch.cat([self.gamma_DFF[0]*x[:,:,0,:,:], self.gamma_DFF[1]*x[:,:,1,:,:], self.gamma_DFF[2]*x[:,:,2,:,:], self.gamma_DFF[3]*y], 1)
       
        y = self.reduceD_DFF(y)  
        y = self.conv_DFF(y)
                       
        if i == 0:
            localFeats = y
        else:
            y = torch.cat([self.gamma_FCF[0]*y, self.gamma_FCF[1]*localFeats], 1) 
            y = self.reduceD_FCF(y)                    
            y = self.conv_FCF(y) 
            localFeats = y  
        y = torch.add(y, skip_y)
        y = self.TwoTail(y) 
        y = y.squeeze(1)   
                
        return y, localFeats  


- ThreeHead aplica convoluciones 3D a la información espectral y TwoHead aplica convoluciones 2D a la información espacial.
- La red pasa por n_module (en este caso 5), donde cada bloque procesa a los datos aplicando convoluciones y atención a los datos.
- Después del procesamiento en bloques, channelX y channelY se concatenan a lo largo del eje de canales, formando tensores más grandes con todas las características procesadas.
- reduceD_X y threefusion aplican capas de reducción y fusión a los datos x, lo que combina las características espectrales. reduceD_Y y twofusion hacen lo mismo para los datos y.
- Después se combina las tres primeras dimensiones espectrales de x y y.
- reduceD_DFF y conv_DFF se utilizan para reducir la dimensionalidad de las características combinadas en y.
- Se combinan las características de reduceD_FCF y conv_FCF y se encarga de reducir la dimensión de esta concatenación para crear un tensor ajustado.
- Se suma y con skip_y en una conexión residual para preservar la información inicial de y, y TwoTail se aplica para llevar la salida a la resolución deseada, eliminando dimensiones redundantes con squeeze(1).

### Definición de funciones para el entrenamiento y validación (train.py)

Función para obtener el checkpoint más reciente

In [ ]:
def get_latest_checkpoint(checkpoint_dir):
    checkpoints = [f for f in os.listdir(checkpoint_dir) if f.endswith('.pth')]
    if len(checkpoints) == 0:
        return None
    checkpoints.sort(key=lambda f: int(f.split('_')[-1].split('.')[0]))
    return os.path.join(checkpoint_dir, checkpoints[-1])

### Función de entrenamiento train

In [ ]:
def train(train_loader, optimizer, model, criterion, epoch, device):
    model.train()
    total_loss = 0

    with tqdm(total=len(train_loader), desc=f"Entrenando Época {epoch}/{opt.nEpochs}", unit="batch", leave=False) as pbar:
        for iteration, batch in enumerate(train_loader, 1):
            input, label = batch[0].to(device), batch[1].to(device)

            localFeats = []
            for i in range(input.shape[1]):
                if i == 0:
                    x = input[:, 0:3, :, :]
                    y = input[:, 0, :, :]
                    new_label = label[:, 0, :, :]
                elif i == input.shape[1] - 1:
                    x = input[:, i-2:i+1, :, :]
                    y = input[:, i, :, :]
                    new_label = label[:, i, :, :]
                else:
                    x = input[:, i-1:i+2, :, :]
                    y = input[:, i, :, :]
                    new_label = label[:, i, :, :]

                SR, localFeats = model(x, y, localFeats, i)
                localFeats = localFeats.detach()

                loss = criterion(SR, new_label)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            total_loss += loss.item()
            pbar.update(1)

            if opt.show:
                writer.add_scalar('Train/Loss', loss.item())

    return total_loss / len(train_loader)


- Realizamos una pasada de entrenamiento para cada lote en el train_loader.
- Después se calcula la pérdida de cada paso y se actualizan los pesos del modelo.

### Función de validación val

In [ ]:
def val(val_loader, model, epoch, device):
    model.eval()
    val_psnr = 0

    with tqdm(total=len(val_loader), desc=f"Validando Época {epoch}", unit="batch", leave=False) as pbar:
        for iteration, batch in enumerate(val_loader, 1):
            input, label = batch[0].to(device), batch[1].to(device)
            SR = np.ones((label.shape[1], label.shape[2], label.shape[3])).astype(np.float32)

            localFeats = []
            for i in range(input.shape[1]):
                if i == 0:
                    x = input[:, 0:3, :, :]
                    y = input[:, 0, :, :]
                    new_label = label[:, 0, :, :]
                elif i == input.shape[1] - 1:
                    x = input[:, i-2:i+1, :, :]
                    y = input[:, i, :, :]
                    new_label = label[:, i, :, :]
                else:
                    x = input[:, i-1:i+2, :, :]
                    y = input[:, i, :, :]
                    new_label = label[:, i, :, :]

                output, localFeats = model(x, y, localFeats, i)
                SR[i, :, :] = output.cpu().data[0].numpy()

            val_psnr += PSNR(SR, label.cpu().data[0].numpy())
            pbar.update(1)

    val_psnr = val_psnr / len(val_loader)
    if opt.show:
        writer.add_scalar('Val/PSNR', val_psnr, epoch)

    return val_psnr

Evalúa el modelo en el conjunto de validación y calcula el PSNR.

### Funciones para guardar gráficos y checkpoints

In [ ]:
def save_plots(epoch):
    if not os.path.exists(out_path):
        os.makedirs(out_path)

    plt.figure()
    plt.plot(range(1, len(loss_values) + 1), loss_values, label='Pérdida (Loss)')
    plt.xlabel('Época')
    plt.ylabel('Pérdida')
    plt.title('Pérdida por Época')
    plt.legend()
    plt.savefig(out_path + f'loss_plot_epoch_{epoch}.png')
    plt.close()

    plt.figure()
    plt.plot(range(1, len(psnr_values) + 1), psnr_values, label='PSNR')
    plt.xlabel('Época')
    plt.ylabel('PSNR')
    plt.title('PSNR por Época')
    plt.legend()
    plt.savefig(out_path + f'psnr_plot_epoch_{epoch}.png')
    plt.close()

def save_checkpoint(model, epoch, optimizer):
    model_out_path = os.path.join(checkpoint_dir, f"model_{opt.upscale_factor}_epoch_{epoch}.pth")
    state = {"epoch": epoch, "model": model.state_dict(), "optimizer": optimizer.state_dict()}
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    torch.save(state, model_out_path)

In [ ]:
def main():
    if opt.show:
        global writer
        writer = SummaryWriter(log_dir='logs')

    # Configurar el dispositivo
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Utilizando dispositivo: {device}")

    if opt.cuda and not torch.cuda.is_available():
        raise Exception("No se encontró una GPU disponible, verifica tu configuración.")

    torch.manual_seed(opt.seed)
    cudnn.benchmark = True

    ## Cargar datasets
    #train_set = TrainsetFromFolder('F:/HyperSSR/SFCSR_Modificado/Data/train/' + opt.datasetName + '/' + str(opt.upscale_factor) + '/')
    #train_loader = DataLoader(dataset=train_set, num_workers=opt.threads, batch_size=opt.batchSize, shuffle=True)

    #val_set = ValsetFromFolder('F:/HyperSSR/SFCSR_Modificado/Data/test/' + opt.datasetName + '/' + str(opt.upscale_factor))
    #val_loader = DataLoader(dataset=val_set, num_workers=opt.threads, batch_size=1, shuffle=False)

    # Definir el directorio base como el directorio actual del archivo
    base_dir = os.path.dirname(os.path.abspath("__file__"))
    data_dir = os.path.join(base_dir, 'Data')

    # Cargar datasets con rutas relativas
    train_set = TrainsetFromFolder(os.path.join(data_dir, 'train', opt.datasetName, str(opt.upscale_factor)))
    train_loader = DataLoader(dataset=train_set, num_workers=opt.threads, batch_size=opt.batchSize, shuffle=True)

    val_set = ValsetFromFolder(os.path.join(data_dir, 'test', opt.datasetName, str(opt.upscale_factor)))
    val_loader = DataLoader(dataset=val_set, num_workers=opt.threads, batch_size=1, shuffle=False)

    # Crear el modelo
    model = SFCSR(opt).to(device)
    criterion = nn.L1Loss().to(device)

    # Usar múltiples GPUs si están disponibles
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)

    print('# parameters:', sum(param.numel() for param in model.parameters()))

    # Configurar el optimizador
    optimizer = optim.Adam(model.parameters(), lr=opt.lr, betas=(0.9, 0.999), eps=1e-08)

    # Cargar checkpoint si existe
    start_epoch = opt.start_epoch
    checkpoint = get_latest_checkpoint(checkpoint_dir)
    
    if checkpoint:
        print(f"=> Cargando checkpoint '{checkpoint}'")
        checkpoint = torch.load(checkpoint)
        start_epoch = checkpoint['epoch'] + 1
        model.load_state_dict(checkpoint['model'])
        optimizer.load_state_dict(checkpoint['optimizer'])
    else:
        print("=> No se encontró ningún checkpoint, comenzando desde la época 1")

    # Scheduler para tasa de aprendizaje
    scheduler = MultiStepLR(optimizer, milestones=[35, 70, 105, 140, 175], gamma=0.5, last_epoch=start_epoch - 1)

    # Bucle de entrenamiento
    for epoch in range(start_epoch, opt.nEpochs + 1):
        scheduler.step()
        print(f"Epoch = {epoch}, lr = {optimizer.param_groups[0]['lr']}")
        train_loss = train(train_loader, optimizer, model, criterion, epoch, device)
        val_psnr = val(val_loader, model, epoch, device)

        loss_values.append(train_loss)
        psnr_values.append(val_psnr)

        save_checkpoint(model, epoch, optimizer)
        print(f"Epoch [{epoch}/{opt.nEpochs}] - PSNR: {val_psnr:.3f}")

        if epoch % 10 == 0:
            save_plots(epoch)

if __name__ == "__main__":
    main()